In [6]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()	

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self,inputs):
        self.inputs=inputs
        exp_values=np.exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probabilities=exp_values/np.sum(exp_values,axis=1,keepdims=True)
        self.output=probabilities
    def backward(self,dvalues):
        self.dinputs=np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output)-np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix,single_dvalues)

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self,y_pred,y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape)==2:
            correct_confidences=np.sum(y_pred_clipped*y_true,axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

def backward(self,dvalues,y_true):
    samples = len(dvalues)
    labels = len(dvalues[0])
    if len(y_true.shape)==1:
        y_true=np.eye(labels)[y_true]
        self.dinputs=-y_true/dvalues
        self.dinputs=self.dinputs/samples

class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation=Activation_Softmax()
        self.loss=Loss_CategoricalCrossentropy()
    def forward(self,inputs,y_true):
        self.activation.forward(inputs)
        self.output=self.activation.output
        return self.loss.calculate(self.output, y_true)
    def backward(self,dvalues,y_true):
        samples=len(dvalues)
        if len(y_true.shape)==2:
            y_true=np.argmax(y_true,axis=1)
        self.dinputs=dvalues.copy()
        self.dinputs[range(samples),y_true]-=1
        self.dinputs=self.dinputs/samples

class Optimizer_SGD:
    def __init__(self,learning_rate=1.0):
        self.learning_rate=learning_rate
    
    def update_params(self,layer):
        layer.weights+=-self.learning_rate*layer.dweights
        layer.biases+=-self.learning_rate*layer.dbiases
        





X, y=spiral_data(samples=100,classes=3)

dense1=Layer_Dense(2,64)
activation1=Activation_ReLU()
dense2=Layer_Dense(64,3)
loss_activation=Activation_Softmax_Loss_CategoricalCrossentropy()

optimizer=Optimizer_SGD()

for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss=loss_activation.forward(dense2.output,y)

    predictions=np.argmax(loss_activation.output,axis=1)
    if len(y.shape)==2:
        y=np.argmax(y,axis=1)
    accuracy=np.mean(predictions==y)
    if not epoch % 100:
        print(f'epoch:{epoch}, '+
              f'acc:{accuracy:.3f}, '+
              f'loss:{loss:.3f}') 
        
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)

epoch:0, acc:0.360, loss:1.099
epoch:100, acc:0.400, loss:1.087
epoch:200, acc:0.417, loss:1.077
epoch:300, acc:0.413, loss:1.076
epoch:400, acc:0.400, loss:1.074
epoch:500, acc:0.400, loss:1.071
epoch:600, acc:0.417, loss:1.067
epoch:700, acc:0.437, loss:1.062
epoch:800, acc:0.423, loss:1.055
epoch:900, acc:0.387, loss:1.064
epoch:1000, acc:0.400, loss:1.063
epoch:1100, acc:0.443, loss:1.063
epoch:1200, acc:0.403, loss:1.061
epoch:1300, acc:0.390, loss:1.053
epoch:1400, acc:0.447, loss:1.060
epoch:1500, acc:0.420, loss:1.043
epoch:1600, acc:0.430, loss:1.032
epoch:1700, acc:0.403, loss:1.048
epoch:1800, acc:0.450, loss:1.041
epoch:1900, acc:0.427, loss:1.025
epoch:2000, acc:0.417, loss:1.038
epoch:2100, acc:0.463, loss:1.021
epoch:2200, acc:0.490, loss:1.007
epoch:2300, acc:0.450, loss:1.002
epoch:2400, acc:0.480, loss:0.994
epoch:2500, acc:0.467, loss:0.994
epoch:2600, acc:0.497, loss:0.975
epoch:2700, acc:0.443, loss:1.002
epoch:2800, acc:0.523, loss:0.965
epoch:2900, acc:0.533, los